In [1]:
print("___ Import Module ___")

import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt

___ Import Module ___
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
os.chdir('./T-Matsuura_LDLT')
path = os.getcwd()

# Fig1C

In [6]:
train_X_180 = pd.read_csv(path+'/Data/train_X_180.csv',index_col=0)
train_y_180 = pd.read_csv(path+'/Data/train_y_180.csv',index_col=0)
val_X_180 = pd.read_csv(path+'/Data/val_X_180.csv',index_col=0)
val_Y_180 = pd.read_csv(path+'/Data/val_Y_180.csv',index_col=0)

In [ ]:
#Unnecessary factor exclusion
X_180 = train_X_180.drop(['Rp_neutro','Rp_lymph','Portal_vein_blood_flow','SLV','Rp_HBsAb_No','graft2_Right lobe and above','graft2_Posterior area','BD stenosis_Yes','Rp_preportal_vein_thrombosis_complete','Rp_preportal_vein_thrombosis_partial','Rp_preportal_vein_thrombosis_No','ALF_Yes','BD stenosis_No','Bile leak_No','Simulect_No','introductory_MMF_No','Biliary stent_No','Bypass_No','shunt_0','Rp_preoperative_esophageal_varices_(post-EIS_also_referred_to_as_EIS)_No','Liver_resection_transplant_history_No','History_of _pper_abdominal_laparotomy_No','Rp_splenectomy1_0','preCKD_No','Rp_preascite_0','preDM_No','portal_vein_thrombosis2_No','graft2_Left lobe','donor_sex_F','bloodrelation_No','HCVAB_No','donor_HTLV1_No','HTLV1_No','donor_HBcAb_No','Rp_sex_F','HCC_No','ALF_No'],axis = 1).reset_index(drop=True)#相関の高い因子の除去
X_180_val = val_X_180.drop(['Rp_neutro','Rp_lymph','Portal_vein_blood_flow','SLV','Rp_HBsAb_No','graft2_Right lobe and above','graft2_Posterior area','BD stenosis_Yes','Rp_preportal_vein_thrombosis_complete','Rp_preportal_vein_thrombosis_partial','Rp_preportal_vein_thrombosis_No','ALF_Yes','BD stenosis_No','Bile leak_No','Simulect_No','introductory_MMF_No','Biliary stent_No','Bypass_No','shunt_0','Rp_preoperative_esophageal_varices_(post-EIS_also_referred_to_as_EIS)_No','Liver_resection_transplant_history_No','History_of _pper_abdominal_laparotomy_No','Rp_splenectomy1_0','preCKD_No','Rp_preascite_0','preDM_No','portal_vein_thrombosis2_No','graft2_Left lobe','donor_sex_F','bloodrelation_No','HCVAB_No','donor_HTLV1_No','HTLV1_No','donor_HBcAb_No','Rp_sex_F','HCC_No','ALF_No'],axis = 1).reset_index(drop=True)#相関の高い因子の除去
y = train_y_180['graftloss_within_6months'].astype('bool').reset_index(drop=True)
val_y = val_Y_180['graftloss_within_6months'].astype('bool').reset_index(drop=True)

In [ ]:
#Parameter tuning
from sklearn.metrics import make_scorer, roc_auc_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE

classifier = RandomForestClassifier(random_state = 4,class_weight='balanced')

#SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
X_resampled, y_resampled = smote.fit_resample(X_180, y)

roc_auc_scorer = make_scorer(roc_auc_score)

#Prameter
param_grid = {
    'n_estimators': [10,50,100, 500,1000],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
    
}


grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring = roc_auc_scorer ,cv=5,verbose=2)
grid_search.fit(X_resampled, y_resampled)
#Show best parameter and score
print("Best parameters found with ROC-AUC: ", grid_search.best_params_)
print("Best ROC-AUC score found : ", grid_search.best_score_)

In [ ]:

# 予測精度を視覚的に評価するために、ROCカーブをプロットする
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import auc, RocCurveDisplay
from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Circle
import numpy as np
#cv = StratifiedKFold(n_splits=4)


tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(12,12))
ax.set_facecolor('white')
smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用

X_resampled, y_resampled = smote.fit_resample(X_180, y)

classifier = RandomForestClassifier(random_state = 4,class_weight='balanced',max_depth =  None,min_samples_split = 2, n_estimators = 500)
classifier.fit(X_resampled, y_resampled)
viz = RocCurveDisplay.from_estimator(
    classifier,
    X_180_val,
    val_y,
    name="AUC",
    alpha=1,
    lw=2,
    ax=ax,color="black"
)


ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="black", label="Chance", alpha=0.8)



ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="AUC = 0.94",
)
plt.axis('square')
custom_legend = [Line2D([0], [0], color='black', linestyle='-', label='ROC', linewidth=2),Line2D([0], [0], color='black', linestyle='--', label='Chance', linewidth=2)]
plt.legend(handles=custom_legend, loc=(0.5,0.02),facecolor='white',prop={ "weight":"normal",'size':32},frameon=False)

        
plt.xlabel('False Positive Rate',weight='normal',size = 48)
plt.ylabel('True Positive Rate',weight='normal',size = 48)
plt.title('AUC = 0.94',weight = 'bold',fontsize= 60)

plt.xticks(fontsize=50,weight='normal')
plt.yticks(fontsize=50,weight='normal')
ax.spines['right'].set_color('white')  # X軸
ax.spines['top'].set_color('white')
plt.rcParams["font.family"] = "Arial"
plt.savefig(path+'Output/Fig1/Fig1C.png', bbox_inches="tight")
# Caluculate score (precision,recall,F1)
y_pred_ = classifier.predict(X_180_val)    
probs = classifier.predict_proba(X_180_val)[:, 1]
fpr, tpr, thresholds = roc_curve(val_y, probs)
auc_score = roc_auc_score(val_y, probs)
print(auc_score)
best_threshold_idx = np.argmax(tpr - fpr)
best_threshold = thresholds[best_threshold_idx]
print("Best threshold:", best_threshold)
saipredictions = (probs >= best_threshold).astype(int)
print(classification_report(val_y, saipredictions))


# Fig1D

In [ ]:
print("___ Import Module ___")
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import auc, roc_auc_score, RocCurveDisplay
import shap
import collections

def dict_addition(original_dict:dict,add_dict:dict):
    for k in add_dict.keys():
        try:
            original_dict[k] += add_dict[k]
        except:
            original_dict[k] = add_dict[k]
    return original_dict

def cvShapFeatureSelect(model,X:pd.DataFrame,y:pd.DataFrame,n_cv:int,):
    # Feature selection by using cv
    cv = StratifiedKFold(n_splits=n_cv)
    max_auc_list,max_j_feature_list = [], []
    for i, (train, test) in enumerate(cv.split(X, y)):
        print("cv:",i)
        # split data
        x_train, x_test, y_train, y_test = X.iloc[train,:],X.iloc[test,:],y.iloc[train],y.iloc[test]
        smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
        X_resampled, y_resampled = smote.fit_resample(x_train, y_train)
        # Generate shap value ranking
        model.fit(X_resampled,y_resampled)
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(x_train)
        shapvalDF = pd.DataFrame({
            "Feature":X.columns,
            "Value":np.abs(shap_values[1]).mean(axis=0)
        })
        shapvalDF = shapvalDF.sort_values("Value",ascending=False)
        
        # Add used feature and decide the number of features that could achieve highest roc-auc value.
        j = 0
        roc_auc_list = []
        while True:
            print("The number of features: ",j+1)
            feature_list_j = shapvalDF["Feature"].values[0:j+1]

            model.fit(X_resampled.loc[:,feature_list_j], y_resampled)
            y_pred = model.predict_proba(x_test.loc[:,feature_list_j])[:,1]
            
            roc_auc_j = roc_auc_score(y_test,y_pred)
            print(roc_auc_j)
            roc_auc_list.append(roc_auc_j)

            # if j>2 and the AUC value drops for two consecutive times, then break the loop
            if j>2:
                if (roc_auc_list[j]<=roc_auc_list[j-1])&((roc_auc_list[j-1]<=roc_auc_list[j-2])):
                    break
            j += 1
        
        max_j = roc_auc_list.index(max(roc_auc_list))
        max_j_features = shapvalDF["Feature"].values[0:max_j+1]
        max_auc_list.append(max(roc_auc_list))
        max_j_feature_list.extend(max_j_features)
        
    
    selected_dict = collections.Counter(max_j_feature_list)

    return np.mean(max_auc_list), np.std(max_auc_list), dict(selected_dict)


def drawROC_changeSplitSeed(model,X:pd.DataFrame,y:pd.DataFrame,x_val:pd.DataFrame,y_val:pd.DataFrame,FeatureSelect:bool,random_seed=1234,n_cv=5,n_loop=1,test_size=0.25, figsize=(5,5), filename=np.nan):
    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    aucs = []
    selected_dict_sum = dict()

    fig, ax = plt.subplots(figsize=figsize)
    for i_seed in range(n_loop):
        print("loop:",i_seed)
        
        
        if FeatureSelect==True:
            max_auc, std_auc, selected_dict = cvShapFeatureSelect(model,X,y,n_cv=n_cv)
            print("CV Max AUROC:",max_auc,"+-",std_auc)
            print("Feature freaquency:",selected_dict)
            X = X.loc[:,selected_dict.keys()]
            x_val = x_val.loc[:,selected_dict.keys()]

        selected_dict_sum = dict_addition(selected_dict_sum,selected_dict)
        smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
        X_resampled, y_resampled = smote.fit_resample(X, y)

        model.fit(X_resampled,y_resampled)
        viz = RocCurveDisplay.from_estimator(
                model,
                x_val,
                y_val,
                name="ROC",
                alpha=0.3,
                lw=1,
                ax=ax
            )
        
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
    plt.close()

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)

    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="black",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="ROC: %0.2f" % (mean_auc),
        xlabel="False Positive Rate",
        ylabel="True Positive Rate"
    )
    ax.legend(fontsize=12)
    plt.axis('square')
    
    if filename != np.nan:
        fig.savefig(filename,bbox_inches="tight")
        plt.close()
    else:
        plt.show()
        plt.close()
    
    return mean_auc,std_auc,selected_dict_sum
    





In [ ]:
if __name__ == "__main__":
    #---------------------------#
    # new dataset
    #---------------------------#    
    X = X_180
    x_val = X_180_val
    y_val = val_y
    y = y

    rfc = RandomForestClassifier(random_state = 4,class_weight='balanced',max_depth =  None,min_samples_split = 2, n_estimators = 500)
    mean_auc,std_auc,selected_feature_dict = drawROC_changeSplitSeed(model=rfc,X=X,y=y,x_val = X_180_val,y_val = val_y,FeatureSelect=True,random_seed=1234,filename=np.nan)
    print(f"Finish! ROCAUC: %0.2f $\pm$ %0.2f)"%(mean_auc, std_auc))
    print("The number of selection of each feature: ",selected_feature_dict)

    selected_feature_DF = pd.DataFrame({"Feature":selected_feature_dict.keys(),"Frequency":selected_feature_dict.values()}).sort_values(["Frequency"],ascending=False)
    print("The number of selection of each feature (as DF): ",selected_feature_DF)

In [ ]:
#列名を論文表記
X_180_7 = X.rename(columns = {'ascite_30POD_':'ascites(30POD)','ascite_14POD_':'ascites(14POD)',
'T_BIL_14POD_':'T-BIL(14POD)','PT_per14POD_':'PT%(14POD)','precondition_Home':'precondition Home',
'donor_max_TB':'D max T-BIL','donor_technique_Right lobectomy':'Right lobectomy'})
X_shap = X_180_7[['ascites(30POD)','ascites(14POD)',
'T-BIL(14POD)','PT%(14POD)','precondition Home',
'D max T-BIL','Right lobectomy']]

In [ ]:
#再定義した列名でSHAP
import shap
import matplotlib.pyplot as plt
classifier = RandomForestClassifier(random_state = 4,class_weight='balanced',max_depth = None, min_samples_split =  2, n_estimators = 500)

smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
X_resampled, y_resampled = smote.fit_resample(X_shap, y)

clf = classifier.fit(X_resampled,y_resampled)
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_shap)
shap.summary_plot(shap_values[1], features= X_shap, show=False, plot_size=(12, 8), max_display=15)

plt.xlabel('                SHAP value',weight='normal',size = 28)
plt.xticks([-0.40,-0.20,0,0.20],fontsize=40,weight='normal')
plt.yticks(fontsize=34,weight='normal')
ax.spines['bottom'].set_color('black')  # X軸
plt.rcParams['font.family'] = 'Arial'
plt.savefig(path+'/Output/Fig1/Fig1D.png',bbox_inches='tight')



# Fig1E

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, RocCurveDisplay
from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Circle

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(12,12))
ax.set_facecolor('white')
smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
X = X_180.loc[:,selected_feature_DF.reset_index(drop=True).loc[0:6,'Feature']]
X_resampled, y_resampled = smote.fit_resample(X, y)

classifier = RandomForestClassifier(random_state = 4,class_weight='balanced',max_depth =  None,min_samples_split = 2, n_estimators = 500)
classifier.fit(X_resampled, y_resampled)
viz = RocCurveDisplay.from_estimator(
    classifier,
    X_180_val.loc[:,selected_feature_DF.reset_index(drop=True).loc[0:6,'Feature']],
    val_y,
    name="AUC",
    alpha=1,
    lw=2,
    ax=ax,color="black"
)

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="black", label="Chance", alpha=0.8)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="AUC = 0.94",
)
plt.axis('square')
custom_legend = [Line2D([0], [0], color='black', linestyle='-', label='ROC', linewidth=2),Line2D([0], [0], color='black', linestyle='--', label='Chance', linewidth=2)]
plt.legend(handles=custom_legend, loc=(0.5,0.02),facecolor='white',prop={ "weight":"normal",'size':32},frameon=False)

        
plt.xlabel('False Positive Rate',weight='normal',size = 48)
plt.ylabel('True Positive Rate',weight='normal',size = 48)
plt.title('AUC = 0.94',weight = 'bold',fontsize= 60)

plt.xticks(fontsize=50,weight='normal')
plt.yticks(fontsize=50,weight='normal')
ax.spines['right'].set_color('white')  # X軸
ax.spines['top'].set_color('white')
plt.rcParams["font.family"] = "Arial"
plt.savefig(path+'Output/Fig1/Fig1E.pdf', bbox_inches="tight")
    
# Caluculate score (precision,recall,F1)
probs = classifier.predict_proba(X_180_val.loc[:,selected_feature_DF.reset_index(drop=True).loc[0:6,'Feature']])[:, 1]

fpr, tpr, thresholds = roc_curve(val_y, probs)

auc_score = roc_auc_score(val_y, probs)
print(auc_score)

best_threshold_idx = np.argmax(tpr - fpr)
best_threshold = thresholds[best_threshold_idx]

print("Best threshold:", best_threshold)
saipredictions = (probs >= best_threshold).astype(int)
print(classification_report(val_y, saipredictions))

# Fig1G

In [ ]:
#Excluding graft loss cases within 180 days
train_X_no180 = train_X_180.drop(train_y_180[train_y_180['graftloss_within_6months'] == 1].index)
train_y_no180 = train_y_180.drop(train_y_180[train_y_180['graftloss_within_6months'] == 1].index)
val_Y_no180 = val_Y_180.drop(val_Y_180[val_Y_180['graftloss_within_6months'] == 1].index)
val_x_no180 = val_X_180.drop(val_Y_180[val_Y_180['graftloss_within_6months'] == 1].index)

In [ ]:
#Unnecessary factor exclusion
X_no180 = train_X_no180.drop(['Rp_neutro','Rp_lymph','Portal_vein_blood_flow','SLV','Rp_HBsAb_No','graft2_Right lobe and above','graft2_Posterior area','BD stenosis_Yes','Rp_preportal_vein_thrombosis_complete','Rp_preportal_vein_thrombosis_partial','Rp_preportal_vein_thrombosis_No','ALF_Yes','BD stenosis_No','Bile leak_No','Simulect_No','introductory_MMF_No','Biliary stent_No','Bypass_No','shunt_0','Rp_preoperative_esophageal_varices_(post-EIS_also_referred_to_as_EIS)_No','Liver_resection_transplant_history_No','History_of _pper_abdominal_laparotomy_No','Rp_splenectomy1_0','preCKD_No','Rp_preascite_0','preDM_No','portal_vein_thrombosis2_No','graft2_Left lobe','donor_sex_F','bloodrelation_No','HCVAB_No','donor_HTLV1_No','HTLV1_No','donor_HBcAb_No','Rp_sex_F','HCC_No','ALF_No'],axis = 1).reset_index(drop=True)#相関の高い因子の除去
val_no180 = val_x_no180.drop(['Rp_neutro','Rp_lymph','Portal_vein_blood_flow','SLV','Rp_HBsAb_No','graft2_Right lobe and above','graft2_Posterior area','BD stenosis_Yes','Rp_preportal_vein_thrombosis_complete','Rp_preportal_vein_thrombosis_partial','Rp_preportal_vein_thrombosis_No','ALF_Yes','BD stenosis_No','Bile leak_No','Simulect_No','introductory_MMF_No','Biliary stent_No','Bypass_No','shunt_0','Rp_preoperative_esophageal_varices_(post-EIS_also_referred_to_as_EIS)_No','Liver_resection_transplant_history_No','History_of _pper_abdominal_laparotomy_No','Rp_splenectomy1_0','preCKD_No','Rp_preascite_0','preDM_No','portal_vein_thrombosis2_No','graft2_Left lobe','donor_sex_F','bloodrelation_No','HCVAB_No','donor_HTLV1_No','HTLV1_No','donor_HBcAb_No','Rp_sex_F','HCC_No','ALF_No'],axis = 1).reset_index(drop=True)#相関の高い因子の除去
y_no180 = train_y_no180['graftloss'].astype('bool').reset_index(drop=True)
val_y_no180 = val_Y_no180['graftloss'].astype('bool').reset_index(drop=True)

In [ ]:
X_early_imp = X_no180.loc[:,selected_feature_DF.reset_index(drop=True).loc[0:6,'Feature']]
val_earlyimp = val_no180.loc[:,selected_feature_DF.reset_index(drop=True).loc[0:6,'Feature']]

In [ ]:
#Parameter tuning for long-term prediction
from sklearn.metrics import make_scorer, roc_auc_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE

classifier = RandomForestClassifier(random_state = 4,class_weight='balanced')

smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
X_resampled, y_resampled = smote.fit_resample(X_no180, y_no180)

roc_auc_scorer = make_scorer(roc_auc_score)

param_grid = {
    'n_estimators': [10,50,100, 500,1000],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
    
}


grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring = roc_auc_scorer ,cv=5,verbose=2)
grid_search.fit(X_resampled, y_resampled)

print("Best parameters found with ROC-AUC: ", grid_search.best_params_)
print("Best ROC-AUC score found : ", grid_search.best_score_)

In [ ]:
#Parameter tuning for short-term prediction

classifier = RandomForestClassifier(random_state = 4,class_weight='balanced')

smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
X_resampled, y_resampled = smote.fit_resample(X_early_imp, y_no180)

roc_auc_scorer = make_scorer(roc_auc_score)

param_grid = {
    'n_estimators': [10,50,100, 500,1000],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]   
}
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring = roc_auc_scorer ,cv=5,verbose=2)
grid_search.fit(X_resampled, y_resampled)

print("Best parameters found with ROC-AUC: ", grid_search.best_params_)
print("Best ROC-AUC score found : ", grid_search.best_score_)

In [ ]:
# 予測精度を視覚的に評価するために、ROCカーブをプロットする
classifier1 = RandomForestClassifier(n_estimators=500,random_state=4,class_weight='balanced', max_depth = None , min_samples_split = 2)

mean_fpr = np.linspace(0, 1, 100)

#Long-term prediction using important features for early graft loss
fig, ax = plt.subplots(figsize=(10,10))
ax.set_facecolor('white')
smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
X_resampled, y_resampled = smote.fit_resample(X_early_imp, y_no180)
classifier1.fit(X_resampled, y_resampled)
viz = RocCurveDisplay.from_estimator(
    classifier1,
    val_earlyimp,
    val_y_no180,
    name="AUC",
    alpha=1,
    lw=2,
    ax=ax,color='b'
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="AUC = 0.95",
)

classifier = RandomForestClassifier(n_estimators=1000,random_state=4,class_weight='balanced', max_depth = None , min_samples_split = 2)
#Long-term prediciton using all features
smote = SMOTE(sampling_strategy='auto', random_state=42)# SMOTEを適用
X_resampled, y_resampled = smote.fit_resample(X_no180, y_no180)
classifier.fit(X_resampled, y_resampled)
viz = RocCurveDisplay.from_estimator(
    classifier,
    val_no180,
    val_y_no180,
    name="AUC",
    alpha=1,
    lw=2,
    ax=ax,color='black'
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="AUC = 0.95",
)



ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="black", label="Chance", alpha=0.8)

plt.axis('square')
custom_legend = [Line2D([0], [0], color='blue', linestyle='-', label='Seven (0.59)', linewidth=2),
                 Line2D([0], [0], color='black', linestyle='-', label='All       (0.78)', linewidth=2),
                 Line2D([0], [0], color='black', linestyle='--', label='Chance', linewidth=2)]

plt.legend(handles=custom_legend, loc=(0.35,0.02),facecolor='white',prop={ "weight":"normal",'size':32},frameon=False)
for text in ax.legend_.texts:
    parts = text.get_text().split('(')
    if len(parts) >= 2:
        text.set_text(parts[0] + ' (' + parts[1])

plt.xlabel('False Positive Rate',weight='normal',size = 42)
plt.ylabel('True Positive Rate',weight='normal',size = 42)
plt.title('ROC curve',weight = 'bold',fontsize= 42,color='white')
plt.xticks(fontsize=42,weight='normal')
plt.yticks(fontsize=42,weight='normal')
ax.spines['top'].set_color('white')  # X軸
ax.spines['right'].set_color('white')
plt.rcParams['font.family'] = 'Arial'
plt.savefig(path+'Output/Fig1/Fig1G.pdf', bbox_inches="tight")